In [1]:
start.time <- Sys.time()

In [5]:
# ---------- Script ----------
# input directory
input_dir <- getwd()
input_dir

[1] "/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/mtbls709re"

In [9]:
# load the functions file
source("/Users/mahnoorzulfiqar/OneDriveUNI/MAW/R/Workflow_R_Functions.r")

In [6]:
#' Import the data
sps_all <- Spectra("/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/mtbls709re/pos_ms2_spectra.mgf", source = MsBackendMgf())

Start data import from 1 files ... 
done



In [7]:
export(sps_all, backend = MsBackendMzR(), file = "/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/mtbls709re/pos_ms2_spectra.mzML")

R_zmq_msg_send errno: 4 strerror: Interrupted system call
R_zmq_msg_send errno: 4 strerror: Interrupted system call




In [8]:
Spectra("/Users/mahnoorzulfiqar/OneDriveUNI/MAW-data/mtbls709re/pos_ms2_spectra.mzML", source = MsBackendMzR())

MSn data (Spectra) with 933 spectra in a MsBackendMzR backend:
      msLevel     rtime scanIndex
    <integer> <numeric> <integer>
1           2  737.5481         1
2           2  724.0025         2
3           2   30.5731         3
4           2   49.7054         4
5           2  723.9392         5
...       ...       ...       ...
929         2  1004.310       929
930         2   940.602       930
931         2   959.586       931
932         2   612.805       932
933         2   598.851       933
 ... 33 more variables/columns.

file(s):
pos_ms2_spectra.mzML

In [15]:
input_table <- data.frame(ms2_rfilename(input_dir))
input_table

mzml_files,ResultFileNames,File_id
<chr>,<chr>,<chr>
./pos_ms2_spectra.mzML,./pos_ms2_spectra,file_1


In [ ]:
for (i in 1:nrow(input_table)){
    
    #Preprocess and Read the mzMLfiles
    spec_pr <- spec_Processing(input_dir,
                               input_table[i, "mzml_files"], 
                               input_table[i, "ResultFileNames"])
    
    # Extract MS2 peak lists
    spec_pr2 <- ms2_peaks(pre_tbl = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"], "/premz_list.txt", sep = ""), "."), sep =""), 
                          proc_mzml = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"], "/processedSpectra.mzML", sep = ""), "."), sep =""),
                          input_dir,
                          result_dir = input_table[i, "ResultFileNames"],
                         file_id = input_table[i, "File_id"])
    
    # camera results for isotopes
#     cam_res <- cam_func(input_dir,
#                         f = input_table[i, "mzml_files"], 
#                         ms2features = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"], "/insilico/MS2DATA.csv", sep = ""), "."), sep = ""))
    
    # Extract MS1 peaks or isotopic peaks
    ms1p <- ms1_peaks(x = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"],'/insilico/MS2DATA.csv', sep = ""), "."), sep =""), 
                      y = NA, 
                      input_table[i, "ResultFileNames"], 
                      input_dir, 
                      QCfile = FALSE)
    
    #prepare sirius parameter files
    sirius_param_files <- sirius_param(x = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"],'/insilico/MS1DATA.csv', sep = ""), "."), sep =""), 
                                       result_dir = input_table[i, 'ResultFileNames'], 
                                       input_dir,
                                       SL = FALSE)
    
    
    # Run sirius
    run_sirius(files = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"],'/insilico/MS1DATA_SiriusP.tsv', sep = ""), "."), sep =""), 
               ppm_max = 5, 
               ppm_max_ms2 = 15, 
               QC = TRUE, 
               SL = FALSE, 
               SL_path = NA,
               candidates = 30,
              profile = "qtof")
}

In [ ]:
end.time <- Sys.time()

In [ ]:
time.taken <- end.time - start.time
print(time.taken)